# Unsupervised Learning: Association Rule Mining


by [__Michael Granitzer__ (michael.granitzer@uni-passau.de)]( http://www.mendeley.com/profiles/michael-granitzer/) and [Konstantin Ziegler (konstantin.ziegler@uni-passau.de)](http://zieglerk.net) based on examples from the [scikit-learn documentation](http://scikit-learn.org/stable/)

__License__

This work is licensed under a [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

1. Turn the pseudo-code for the A-priori algorithm from the lecture into a quick-and-dirty implementation. (Efficiency is not crucial, because the dataset is not that large. Correctness is. An implementation with python lists should suffice, see the end of the notebook.)
2. Use your algorithm from 1. on the dataset to find frequent itemsets. 
Note: The algorithm from the lecture runs only on binary features; So, you have to turn the numerical features into binaries, e.g. by comparison with the average.
Note: This requires the choice of a suitable minimum support; in practice you don't know "suitable" in advance, so binary search the possible values for sigma until the number of frequent itemsets is easy for you to examine by hand.
3. Bonus: Use the frequent itemsets of 2. to find strong association rules. Note: Analogously to 2., this requires the choice of a proper confidence. What is the best possible confidence that you can achieve?
4. Bonus: Filter your association rules for correlation rules, e.g. using lift as correlation measure.
5. Interpretation: Could the observed rule be an effect of a larger cause?

# 1 Implementation of the Apriori-Algorithm

In [117]:
# Example from the Lecture, slide 8
names = ['milk', 'butter', 'coffee', 'cacao', 'cake', 'sugar', 'tea']
X = [[1, 1, 0, 0, 0, 0, 0],
     [1, 0, 1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1, 0, 0],
     [0, 0, 1, 0, 0, 1, 1],
     [1, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1, 1]]
# Some 2-itemsets
I1 = [0, 1]
I2 = [1, 5]
I3 = [0, 2]

In [118]:
x = X[1]
any([x[j] == 1 for j in I1])

True

In [119]:
# Return corresponding instance sets
def is_instance(x, I):
    return all([x[j] == 1 for j in I])

def all_instances(X, I):
    return [x for x in X if is_instance(x, I)]

print(all_instances(X, I1))
print(all_instances(X, I2))
print(all_instances(X, I3))

[[1, 1, 0, 0, 0, 0, 0]]
[]
[[1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0]]


In [120]:
def support(X, I):
    return len(all_instances(X, I))/len(X)
print(support(X, I1))
print(support(X, I2))
print(support(X, I3))


0.16666666666666666
0.0
0.3333333333333333


In [127]:
def is_frequent_itemset(X, L, sigma):
    return all([support(X, I) >= sigma for I in L])

print(I1)
print(all_instances(X, I1))
print(is_frequent_itemset(X, all_instances(X, I1), 1/6))
print(is_frequent_itemset(X, all_instances(X, I1), 1/3))

[0, 1]
[[1, 1, 0, 0, 0, 0, 0]]
True
False


In [128]:
def hasAprioriProperty(c, Llast):
    for item in c:
        d = c[:]
        d.remove(item)
        if d not in Llast:
            return False
    return True

def aPriori(X, sigma, verbose=False):
    L1 = [[i] for i in range(len(X[0])) if support(X, [i]) >= sigma]
    L = [L1]
    for k in range(2, len(X) + 1):
        if verbose:
            print('start loop with k=%d and L=%s' %(k, L))
        C = [sorted(a + b) for a in L[-1] for b in L1 if b[0] not in a]
        C = [c for i, c in enumerate(C) if c not in C[:i]]    # remove duplicates, because we use lists instead of sets
        if verbose:
            print('before Apriori-Check C=%s' %C)
        C = [c for c in C if hasAprioriProperty(c, L[-1])]
        if verbose:
            print('after Apriori-Check C=%s' %C)
        Csigma = dict([(tuple(c), 0.0) for c in C])
        for x in X:
            Ct = [c for c in C if is_instance(x, c)]
            for c in Ct:
                Csigma[tuple(c)] += 1/len(X)
        C = [c for c in C if Csigma[tuple(c)] >= sigma]
        if len(C) == 0:
            break
        L += [C]       
    return [c for Lk in L for c in Lk ]

print(aPriori(X, 0.3))
print(aPriori(X, 0.5))
print(aPriori(X, 0.7))    

[[0], [2], [4], [5], [6], [0, 2], [0, 4], [2, 5], [5, 6]]
[[0], [2], [5]]
[]


In [129]:
# Example from Lecture p. 21
names = ['I1', 'I2', 'I3', 'I4', 'I5']
X     = [[1,    1,    0,    0,    1],
         [0,    1,    0,    1,    0],
         [0,    1,    1,    0,    0],
         [1,    1,    0,    1,    0],
         [1,    0,    1,    0,    0],
         [0,    1,    1,    0,    0],
         [1,    0,    1,    0,    0],
         [1,    1,    1,    0,    1],
         [1,    1,    1,    0,    0]]

print(aPriori(X, 2/9))

[[0], [1], [2], [3], [4], [0, 1], [0, 2], [0, 4], [1, 2], [1, 3], [1, 4], [0, 1, 2], [0, 1, 4]]


# Frequent itemset mining on Boston Housing Data

We'll do frequent itemset mining on the Boston Housing Data. Most of the features are numerical and we'll turn them into Boolean by splitting across the mean/median.

In [136]:
from sklearn.datasets import load_boston
experiment = load_boston()
data, target, feat = experiment.data, experiment.target, experiment.feature_names
print(experiment['DESCR'])

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [137]:
# join data and target for unsupervised learning
target = target.reshape(506, 1)
Xfull = np.concatenate((data, target), axis=1)
np.append(feat, 'MEDV')

print(Xfull[:5, :3])

# turn it into a boolean dataset by comparison with the "average"
avg = Xfull.mean(axis=0)
# from numpy import median
# avg = median(X, axis=0)
X = Xfull > avg

print(avg[:3])
print(X[:5, :3])
print(feat[:3])

[[  6.32000000e-03   1.80000000e+01   2.31000000e+00]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00]
 [  2.72900000e-02   0.00000000e+00   7.07000000e+00]
 [  3.23700000e-02   0.00000000e+00   2.18000000e+00]
 [  6.90500000e-02   0.00000000e+00   2.18000000e+00]]
[  3.59376071  11.36363636  11.13677866]
[[False  True False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]]
['CRIM' 'ZN' 'INDUS']


In [142]:
print(aPriori(X, .4))

[[2], [4], [5], [6], [7], [10], [11], [12], [13], [6, 11], [10, 11]]
